In [724]:
# Install faker if it's not already installed
# !pip install faker

In [725]:
# import pandas as pd
# import numpy as np
# import random
# from faker import Faker
# from datetime import datetime, timedelta, date

# # ==================================================
# # SETUP
# # ==================================================
# np.random.seed(42)
# random.seed(42)
# fake = Faker("en_IN")

# START_DATE = datetime(2022, 1, 1)
# END_DATE = datetime(2025, 12, 31)

# NUM_CUSTOMERS = 20000
# NUM_PRODUCTS = 1000

# # ==================================================
# # 1. PRODUCTS TABLE
# # ==================================================
# categories = ["Electronics", "Grocery", "Fashion", "Home", "Beauty", "Sports"]

# products = []
# for i in range(1, NUM_PRODUCTS + 1):
#     products.append({
#         "product_id": i,
#         "product_name": fake.word().title(),
#         "product_category": random.choice(categories + ["electronics", "ELECTRONICS"]),
#         "unit_price": round(random.uniform(50, 5000), 2),
#         "current_stock_level": random.randint(0, 500)
#     })

# products_df = pd.DataFrame(products)

# # ==================================================
# # 2. CUSTOMER DETAILS
# # ==================================================
# customers = []
# for i in range(1, NUM_CUSTOMERS + 1):
#     since = fake.date_between(
#         start_date=START_DATE - timedelta(days=365 * 3),
#         end_date=END_DATE - timedelta(days=120)
#     )
#     # Convert 'since' from datetime.date to datetime.datetime for consistent type handling
#     if isinstance(since, date) and not isinstance(since, datetime):
#         since = datetime.combine(since, datetime.min.time())

#     customers.append({
#         "customer_id": str(i) if i % 8 == 0 else i,   # datatype inconsistency
#         "first_name": fake.first_name(),
#         "email": None if random.random() < 0.15 else fake.email(),
#         "loyalty_status": random.choice(["Bronze", "Silver", "Gold", None]),
#         "total_loyalty_points": None,
#         "segment_id": None if random.random() < 0.1 else random.randint(1, 4),
#         "customer_phone": None if random.random() < 0.15 else fake.msisdn(),
#         "customer_since": since,
#         "last_purchase_date": None
#     })

# customers_df = pd.DataFrame(customers)

# # ==================================================
# # 3. TRANSACTIONS + LINE ITEMS
# # ==================================================
# transactions = []
# line_items = []

# transaction_id = 1
# line_item_id = 1

# for _, cust in customers_df.iterrows():

#     # Customer behavior type (key for CLV accuracy)
#     behavior = random.choices(
#         ["high", "medium", "low"],
#         weights=[0.25, 0.5, 0.25]
#     )[0]

#     if behavior == "high":
#         num_tx = random.randint(15, 25)
#     elif behavior == "medium":
#         num_tx = random.randint(6, 14)
#     else:
#         num_tx = random.randint(1, 5)

#     last_tx_date = cust["customer_since"]

#     for _ in range(num_tx):
#         gap_days = random.randint(7, 45)
#         tx_date = last_tx_date + timedelta(days=gap_days)

#         if tx_date > END_DATE:
#             break

#         transactions.append({
#             "transaction_id": str(transaction_id) if transaction_id % 11 == 0 else transaction_id,
#             "customer_id": cust["customer_id"],
#             "store_id": random.randint(1, 50),
#             "transaction_date": tx_date,
#             "total_amount": None,
#             "customer_phone": cust["customer_phone"]
#         })

#         basket_total = 0
#         num_items = random.randint(1, 5)

#         for _ in range(num_items):
#             prod = products_df.sample(1).iloc[0]
#             qty = random.randint(1, 3)
#             amount = prod["unit_price"] * qty

#             # refund / noise
#             if random.random() < 0.02:
#                 amount *= -1

#             line_items.append({
#                 "line_item_id": line_item_id,
#                 "transaction_id": transactions[-1]["transaction_id"],
#                 "product_id": prod["product_id"] if random.random() > 0.02 else 999999,
#                 "promotion_id": None if random.random() < 0.7 else random.randint(1, 25),
#                 "line_item_amount": round(amount, 2)
#             })

#             basket_total += amount
#             line_item_id += 1

#         transactions[-1]["total_amount"] = round(basket_total, 2)

#         transaction_id += 1
#         last_tx_date = tx_date

# # ==================================================
# # DATAFRAMES
# # ==================================================
# transactions_df = pd.DataFrame(transactions)
# line_items_df = pd.DataFrame(line_items)

# # ==================================================
# # REAL-WORLD DATA ISSUES (INTENTIONAL)
# # ==================================================

# # Duplicate customers
# customers_df = pd.concat(
#     [customers_df, customers_df.sample(400)],
#     ignore_index=True
# )

# # Duplicate transactions
# transactions_df = pd.concat(
#     [transactions_df, transactions_df.sample(1200)],
#     ignore_index=True
# )

# # last_purchase_date (leakage candidate)
# last_dates = transactions_df.groupby("customer_id")["transaction_date"].max()
# customers_df["last_purchase_date"] = customers_df["customer_id"].map(last_dates)

# # loyalty points derived from spend
# total_spend = transactions_df.groupby("customer_id")["total_amount"].sum()
# customers_df["total_loyalty_points"] = customers_df["customer_id"].map(
#     lambda x: int(total_spend.get(x, 0) / 100)
# )

# # ==================================================
# # SAVE FILES
# # ==================================================
# customers_df.to_csv("customer_details.csv", index=False)
# products_df.to_csv("products.csv", index=False)
# transactions_df.to_csv("store_sales_header.csv", index=False)
# line_items_df.to_csv("store_sales_line_items.csv", index=False)

# print("✅ FINAL DATASET GENERATED SUCCESSFULLY")
# print("Customers:", len(customers_df))
# print("Products:", len(products_df))
# print("Transactions:", len(transactions_df))
# print("Line Items:", len(line_items_df))


Importing data into dataframe using Pandas

In [726]:
import pandas as pd


In [727]:
customer_df = pd.read_csv('customer_details.csv')

In [728]:
product_df = pd.read_csv('products.csv')

In [729]:
store_sales_header_df = pd.read_csv('store_sales_header.csv')

In [730]:
store_sales_line_items_df= pd.read_csv('store_sales_line_items.csv')

Date ensureing

In [731]:
# =====================================
# Ensure date columns are datetime
# =====================================
store_sales_header_df["transaction_date"] = pd.to_datetime(
    store_sales_header_df["transaction_date"], errors="coerce"
)

customer_df["customer_since"] = pd.to_datetime(
    customer_df["customer_since"], errors="coerce"
)

customer_df["last_purchase_date"] = pd.to_datetime(
    customer_df["last_purchase_date"], errors="coerce"
)

Knowing About Data

In [732]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20400 entries, 0 to 20399
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           20400 non-null  int64         
 1   first_name            20400 non-null  object        
 2   email                 17267 non-null  object        
 3   loyalty_status        15210 non-null  object        
 4   total_loyalty_points  20400 non-null  int64         
 5   segment_id            18390 non-null  float64       
 6   customer_phone        17212 non-null  float64       
 7   customer_since        20400 non-null  datetime64[ns]
 8   last_purchase_date    20400 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(3)
memory usage: 1.4+ MB


In [733]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           1000 non-null   int64  
 1   product_name         1000 non-null   object 
 2   product_category     1000 non-null   object 
 3   unit_price           1000 non-null   float64
 4   current_stock_level  1000 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB


In [734]:
store_sales_header_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206028 entries, 0 to 206027
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_id    206028 non-null  int64         
 1   customer_id       206028 non-null  int64         
 2   store_id          206028 non-null  int64         
 3   transaction_date  206028 non-null  datetime64[ns]
 4   total_amount      206028 non-null  float64       
 5   customer_phone    173139 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 9.4 MB


In [735]:
store_sales_line_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614457 entries, 0 to 614456
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   line_item_id      614457 non-null  int64  
 1   transaction_id    614457 non-null  int64  
 2   product_id        614457 non-null  int64  
 3   promotion_id      184341 non-null  float64
 4   line_item_amount  614457 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 23.4 MB


In [736]:
# =====================================
# 1. NULL VALUE CHECK
# =====================================
print("\n🔹 NULL VALUES CHECK")

print("\nCustomer Table Nulls:")
print(customer_df.isnull().sum())

print("\nStore Sales Header Nulls:")
print(store_sales_header_df.isnull().sum())

print("\nLine Items Nulls:")
print(store_sales_line_items_df.isnull().sum())

print("\nProduct Table Nulls:")
print(product_df.isnull().sum())


🔹 NULL VALUES CHECK

Customer Table Nulls:
customer_id                0
first_name                 0
email                   3133
loyalty_status          5190
total_loyalty_points       0
segment_id              2010
customer_phone          3188
customer_since             0
last_purchase_date         0
dtype: int64

Store Sales Header Nulls:
transaction_id          0
customer_id             0
store_id                0
transaction_date        0
total_amount            0
customer_phone      32889
dtype: int64

Line Items Nulls:
line_item_id             0
transaction_id           0
product_id               0
promotion_id        430116
line_item_amount         0
dtype: int64

Product Table Nulls:
product_id             0
product_name           0
product_category       0
unit_price             0
current_stock_level    0
dtype: int64


Null Values Imputation

In [737]:
# ==============================
# 1. CUSTOMER TABLE
# ==============================

# Drop non-predictive columns
customer_df = customer_df.drop(columns=["email", "customer_phone"])

# Fill categorical nulls
customer_df["segment_id"] = customer_df["segment_id"].fillna("Unknown")
customer_df["loyalty_status"] = customer_df["loyalty_status"].fillna("Unknown")

# Fill numeric nulls
customer_df["total_loyalty_points"] = customer_df["total_loyalty_points"].fillna(0)

# Do NOT touch last_purchase_date yet (leakage sensitive)

# ==============================
# 2. PRODUCT TABLE
# ==============================

product_df["product_category"] = product_df["product_category"].fillna("Unknown")

# Drop products with no price (rare but dangerous)
product_df = product_df.dropna(subset=["unit_price"])

# Fill stock with median
product_df["current_stock_level"] = product_df["current_stock_level"].fillna(
    product_df["current_stock_level"].median()
)

# ==============================
# 3. STORE SALES HEADER
# ==============================

# Drop non-predictive column
store_sales_header_df = store_sales_header_df.drop(columns=["customer_phone"])

# Drop transactions with no date (cannot be used)
store_sales_header_df = store_sales_header_df.dropna(subset=["transaction_date"])

# Leave total_amount untouched (will be recomputed later)

# ==============================
# 4. STORE SALES LINE ITEMS
# ==============================

# Promotion missing means no promotion
store_sales_line_items_df["promotion_id"] = store_sales_line_items_df[
    "promotion_id"
].fillna(-1)

# Drop line items with missing amount
store_sales_line_items_df = store_sales_line_items_df.dropna(
    subset=["line_item_amount"]
)

# ==============================
# FINAL SANITY CHECK
# ==============================

print("✅ NULL HANDLING COMPLETED\n")

print("Customer nulls:\n", customer_df.isnull().sum())
print("\nProduct nulls:\n", product_df.isnull().sum())
print("\nSales header nulls:\n", store_sales_header_df.isnull().sum())
print("\nLine items nulls:\n", store_sales_line_items_df.isnull().sum())


✅ NULL HANDLING COMPLETED

Customer nulls:
 customer_id             0
first_name              0
loyalty_status          0
total_loyalty_points    0
segment_id              0
customer_since          0
last_purchase_date      0
dtype: int64

Product nulls:
 product_id             0
product_name           0
product_category       0
unit_price             0
current_stock_level    0
dtype: int64

Sales header nulls:
 transaction_id      0
customer_id         0
store_id            0
transaction_date    0
total_amount        0
dtype: int64

Line items nulls:
 line_item_id        0
transaction_id      0
product_id          0
promotion_id        0
line_item_amount    0
dtype: int64


Foreign Key Integrity

In [738]:
# # Invalid transaction_id in line items
# invalid_tx_fk_mask = ~store_sales_line_items_df["transaction_id"].isin(
#     store_sales_header_df["transaction_id"]
# )

# print("Line items with invalid transaction_id:",
#       invalid_tx_fk_mask.sum())

# # Invalid product_id in line items
# invalid_product_fk_mask = ~store_sales_line_items_df["product_id"].isin(
#     product_df["product_id"]
# )

# print("Line items with invalid product_id:",
#       invalid_product_fk_mask.sum())


In [739]:
# # Keep only valid line items
# store_sales_line_items_df = store_sales_line_items_df[
#     (~invalid_tx_fk_mask) & (~invalid_product_fk_mask)
# ].copy()

# print("Line items after FK cleanup:",
#       store_sales_line_items_df.shape[0])


PHASE 1: EXPLORATORY DATA ANALYSIS (EDA)


In [740]:
def table_eda(df, table_name, pk=None, date_cols=None):
    print(f"\n================ {table_name.upper()} ================\n")

    # Row & column count
    print("Shape:", df.shape)

    # Column info
    print("\nColumn Info:")
    print(df.dtypes)

    # Null percentage
    print("\nNull Percentage (%):")
    print((df.isnull().mean() * 100).round(2))

    # Duplicate PK check
    if pk:
        print(f"\nDuplicate {pk}:",
              df.duplicated(subset=[pk]).sum())

    # Date ranges
    if date_cols:
        for col in date_cols:
            print(f"\nDate range for {col}:")
            print("Min:", df[col].min())
            print("Max:", df[col].max())

    # Basic numeric stats
    print("\nNumeric Summary:")
    print(df.describe())


In [741]:
table_eda(
    df=customer_df,
    table_name="customer_df",
    pk="customer_id",
    date_cols=["customer_since", "last_purchase_date"]
)



================ CUSTOMER_DF ================

Shape: (20400, 7)

Column Info:
customer_id                      int64
first_name                      object
loyalty_status                  object
total_loyalty_points             int64
segment_id                      object
customer_since          datetime64[ns]
last_purchase_date      datetime64[ns]
dtype: object

Null Percentage (%):
customer_id             0.0
first_name              0.0
loyalty_status          0.0
total_loyalty_points    0.0
segment_id              0.0
customer_since          0.0
last_purchase_date      0.0
dtype: float64

Duplicate customer_id: 400

Date range for customer_since:
Min: 2019-01-02 00:00:00
Max: 2025-09-01 00:00:00

Date range for last_purchase_date:
Min: 2019-01-26 00:00:00
Max: 2025-12-31 00:00:00

Numeric Summary:
        customer_id  total_loyalty_points                 customer_since  \
count  20400.000000          20400.000000                          20400   
mean   10004.161520           1485

In [742]:
table_eda(
    df=product_df,
    table_name="product_df",
    pk="product_id"
)



================ PRODUCT_DF ================

Shape: (1000, 5)

Column Info:
product_id               int64
product_name            object
product_category        object
unit_price             float64
current_stock_level      int64
dtype: object

Null Percentage (%):
product_id             0.0
product_name           0.0
product_category       0.0
unit_price             0.0
current_stock_level    0.0
dtype: float64

Duplicate product_id: 0

Numeric Summary:
        product_id   unit_price  current_stock_level
count  1000.000000  1000.000000          1000.000000
mean    500.500000  2503.754980           250.938000
std     288.819436  1430.497204           137.705348
min       1.000000    57.740000             0.000000
25%     250.750000  1259.677500           140.000000
50%     500.500000  2525.845000           254.000000
75%     750.250000  3747.217500           370.000000
max    1000.000000  4999.540000           500.000000


In [743]:
table_eda(
    df=store_sales_header_df,
    table_name="store_sales_header_df",
    pk="transaction_id",
    date_cols=["transaction_date"]
)



================ STORE_SALES_HEADER_DF ================

Shape: (206028, 5)

Column Info:
transaction_id               int64
customer_id                  int64
store_id                     int64
transaction_date    datetime64[ns]
total_amount               float64
dtype: object

Null Percentage (%):
transaction_id      0.0
customer_id         0.0
store_id            0.0
transaction_date    0.0
total_amount        0.0
dtype: float64

Duplicate transaction_id: 1200

Date range for transaction_date:
Min: 2019-01-09 00:00:00
Max: 2025-12-31 00:00:00

Numeric Summary:
       transaction_id    customer_id       store_id  \
count   206028.000000  206028.000000  206028.000000   
mean    102415.695939    9983.546785      25.542091   
min          1.000000       1.000000       1.000000   
25%      51210.750000    4969.000000      13.000000   
50%     102418.500000    9980.000000      26.000000   
75%     153617.250000   14967.000000      38.000000   
max     204828.000000   20000.000000      50

In [744]:
table_eda(
    df=store_sales_line_items_df,
    table_name="store_sales_line_items_df",
    pk="line_item_id"
)



================ STORE_SALES_LINE_ITEMS_DF ================

Shape: (614457, 5)

Column Info:
line_item_id          int64
transaction_id        int64
product_id            int64
promotion_id        float64
line_item_amount    float64
dtype: object

Null Percentage (%):
line_item_id        0.0
transaction_id      0.0
product_id          0.0
promotion_id        0.0
line_item_amount    0.0
dtype: float64

Duplicate line_item_id: 0

Numeric Summary:
        line_item_id  transaction_id     product_id   promotion_id  \
count  614457.000000    614457.00000  614457.000000  614457.000000   
mean   307229.000000    102364.44575   20639.680267       3.200021   
std    177378.601514     59103.19833  140441.518555       7.532419   
min         1.000000         1.00000       1.000000      -1.000000   
25%    153615.000000     51158.00000     255.000000      -1.000000   
50%    307229.000000    102302.00000     511.000000      -1.000000   
75%    460843.000000    153588.00000     766.000000       5

===============================
1.2 RELATIONSHIP-LEVEL EDA
===============================

In [745]:
customer_tx_counts = store_sales_header_df["customer_id"].value_counts()

print("\nTransactions per Customer Summary:")
print(customer_tx_counts.describe())



Transactions per Customer Summary:
count    20000.000000
mean        10.301400
std          6.309774
min          1.000000
25%          5.000000
50%          9.000000
75%         14.000000
max         27.000000
Name: count, dtype: float64


In [746]:
repeat_customers = (customer_tx_counts > 1).sum()
total_customers = customer_tx_counts.shape[0]

print("\nRepeat Customers (>1 transaction):", repeat_customers)
print("Repeat Customer %:",
      round(100 * repeat_customers / total_customers, 2), "%")



Repeat Customers (>1 transaction): 19042
Repeat Customer %: 95.21 %


In [747]:
line_items_per_tx = store_sales_line_items_df["transaction_id"].value_counts()

print("\nLine Items per Transaction Summary:")
print(line_items_per_tx.describe())



Line Items per Transaction Summary:
count    204828.000000
mean          2.999868
std           1.413273
min           1.000000
25%           2.000000
50%           3.000000
75%           4.000000
max           5.000000
Name: count, dtype: float64


In [748]:
product_usage = store_sales_line_items_df["product_id"].value_counts()

print("\nProduct Usage Summary:")
print(product_usage.describe())



Product Usage Summary:
count     1001.000000
mean       613.843157
std        373.143013
min        532.000000
25%        584.000000
50%        601.000000
75%        619.000000
max      12381.000000
Name: count, dtype: float64


===============================
1.3 DATE & COVERAGE SANITY CHECKS
===============================

In [749]:
print("\nOverall Transaction Date Coverage:")
print("Min:", store_sales_header_df["transaction_date"].min())
print("Max:", store_sales_header_df["transaction_date"].max())



Overall Transaction Date Coverage:
Min: 2019-01-09 00:00:00
Max: 2025-12-31 00:00:00


🔹 Customer lifetime duration

In [750]:
customer_lifetime = (
    store_sales_header_df
    .groupby("customer_id")["transaction_date"]
    .agg(["min", "max"])
)

customer_lifetime["active_days"] = (
    customer_lifetime["max"] - customer_lifetime["min"]
).dt.days

print("\nCustomer Active Period (Days):")
print(customer_lifetime["active_days"].describe())



Customer Active Period (Days):
count    20000.000000
mean       239.658400
std        165.926321
min          0.000000
25%        109.000000
50%        215.000000
75%        341.000000
max        763.000000
Name: active_days, dtype: float64


2.1 FIX DATATYPE INCONSISTENCIES


In [751]:
# IDs → consistent type
customer_df["customer_id"] = customer_df["customer_id"].astype(int)
product_df["product_id"] = product_df["product_id"].astype(int)
store_sales_header_df["transaction_id"] = store_sales_header_df["transaction_id"].astype(int)
store_sales_header_df["customer_id"] = store_sales_header_df["customer_id"].astype(int)
store_sales_line_items_df["transaction_id"] = store_sales_line_items_df["transaction_id"].astype(int)
store_sales_line_items_df["product_id"] = store_sales_line_items_df["product_id"].astype(int)

# Dates
customer_df["customer_since"] = pd.to_datetime(customer_df["customer_since"], errors="coerce")
customer_df["last_purchase_date"] = pd.to_datetime(customer_df["last_purchase_date"], errors="coerce")
store_sales_header_df["transaction_date"] = pd.to_datetime(
    store_sales_header_df["transaction_date"], errors="coerce"
)

# Monetary fields
store_sales_header_df["total_amount"] = pd.to_numeric(
    store_sales_header_df["total_amount"], errors="coerce"
)
store_sales_line_items_df["line_item_amount"] = pd.to_numeric(
    store_sales_line_items_df["line_item_amount"], errors="coerce"
)


2.2 HANDLE NULL VALUES (BUSINESS-SAFE)

In [752]:
# Drop non-predictive columns
customer_df = customer_df.drop(columns=["first_name"])

# Fill categorical nulls and ensure string type
customer_df["segment_id"] = customer_df["segment_id"].fillna("Unknown").astype(str)
customer_df["loyalty_status"] = customer_df["loyalty_status"].fillna("Unknown").astype(str)

# Fill numeric nulls
customer_df["total_loyalty_points"] = customer_df["total_loyalty_points"].fillna(0)

# DO NOT fill last_purchase_date (leakage sensitive)


 Product table

In [753]:
product_df["product_category"] = product_df["product_category"].fillna("Unknown")

# Drop products without price (unsafe for spend modeling)
product_df = product_df.dropna(subset=["unit_price"])

# Stock is operational → median fill
product_df["current_stock_level"] = product_df["current_stock_level"].fillna(
    product_df["current_stock_level"].median()
)


Store sales header

In [754]:
# Drop transactions without date
store_sales_header_df = store_sales_header_df.dropna(subset=["transaction_date"])


Line items

In [755]:
store_sales_line_items_df["promotion_id"] = store_sales_line_items_df["promotion_id"].fillna(-1)

# Drop line items without monetary value
store_sales_line_items_df = store_sales_line_items_df.dropna(subset=["line_item_amount"])


2.3 REMOVE DUPLICATES (PK-BASED)

In [756]:
customer_df = customer_df.drop_duplicates(subset=["customer_id"])
product_df = product_df.drop_duplicates(subset=["product_id"])
store_sales_header_df = store_sales_header_df.drop_duplicates(subset=["transaction_id"])
store_sales_line_items_df = store_sales_line_items_df.drop_duplicates(subset=["line_item_id"])


2.4 ENFORCE FOREIGN KEY INTEGRITY

In [757]:
# Drop line items with invalid transaction_id
store_sales_line_items_df = store_sales_line_items_df[
    store_sales_line_items_df["transaction_id"].isin(
        store_sales_header_df["transaction_id"]
    )
]

# Drop line items with invalid product_id
store_sales_line_items_df = store_sales_line_items_df[
    store_sales_line_items_df["product_id"].isin(
        product_df["product_id"]
    )
]


2.5 FIX DATE ISSUES

In [758]:
# Remove transactions before customer signup
store_sales_header_df = store_sales_header_df.merge(
    customer_df[["customer_id", "customer_since"]],
    on="customer_id",
    how="left"
)

store_sales_header_df = store_sales_header_df[
    store_sales_header_df["transaction_date"] >= store_sales_header_df["customer_since"]
]

store_sales_header_df = store_sales_header_df.drop(columns=["customer_since"])

# Remove future transactions (relative to dataset snapshot)
snapshot_date = store_sales_header_df["transaction_date"].max()

store_sales_header_df = store_sales_header_df[
    store_sales_header_df["transaction_date"] <= snapshot_date
]


2.6 FIX MONETARY MISMATCHES (CRITICAL)

In [759]:
# Recompute transaction totals from line items
recomputed_totals = (
    store_sales_line_items_df
    .groupby("transaction_id", as_index=False)["line_item_amount"]
    .sum()
    .rename(columns={"line_item_amount": "recomputed_total_amount"})
)

# Replace header total_amount
store_sales_header_df = store_sales_header_df.merge(
    recomputed_totals,
    on="transaction_id",
    how="left"
)

store_sales_header_df["total_amount"] = store_sales_header_df[
    "recomputed_total_amount"
]

store_sales_header_df = store_sales_header_df.drop(
    columns=["recomputed_total_amount"]
)


2.7 HANDLE OUTLIERS (SAFE FOR CLV)

In [760]:
# Customer total spend
customer_spend = store_sales_header_df.groupby(
    "customer_id"
)["total_amount"].sum()

# Log transform (recommended)
customer_spend_log = np.log1p(customer_spend)



/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


2.8 IDENTIFY COLD-START CUSTOMERS

In [761]:
tx_counts = store_sales_header_df["customer_id"].value_counts()

cold_start_customers = tx_counts[tx_counts <= 1].index

customer_df["is_cold_start"] = customer_df["customer_id"].isin(
    cold_start_customers
).astype(int)


FINAL SANITY CHECK

In [762]:
print("✅ PHASE 2 CLEANING COMPLETED\n")

print("Customers:", customer_df.shape)
print("Products:", product_df.shape)
print("Transactions:", store_sales_header_df.shape)
print("Line Items:", store_sales_line_items_df.shape)


✅ PHASE 2 CLEANING COMPLETED

Customers: (20000, 7)
Products: (1000, 5)
Transactions: (204828, 5)
Line Items: (602076, 5)


AGGREGATION

3.1 DEFINE SNAPSHOT DATE

In [763]:
snapshot_date = store_sales_header_df["transaction_date"].max()


3.2 TRANSACTION-LEVEL AGGREGATES

In [764]:
# Base transaction aggregation
txn_agg = (
    store_sales_header_df
    .groupby("customer_id")
    .agg(
        total_spend=("total_amount", "sum"),
        total_transactions=("transaction_id", "count"),
        avg_order_value=("total_amount", "mean"),
        first_transaction_date=("transaction_date", "min"),
        last_transaction_date=("transaction_date", "max")
    )
    .reset_index()
)

# Recency & tenure
txn_agg["recency_days"] = (
    snapshot_date - txn_agg["last_transaction_date"]
).dt.days

txn_agg["tenure_days"] = (
    snapshot_date - txn_agg["first_transaction_date"]
).dt.days


3.3 TIME-WINDOW FEATURES (30 / 60 / 90 DAYS)

In [765]:
def window_agg_safe(df, start_day, end_day, label):
    start = cutoff_date - pd.Timedelta(days=end_day)
    end = cutoff_date - pd.Timedelta(days=start_day)

    temp = df[
        (df["transaction_date"] > start) &
        (df["transaction_date"] <= end)
    ]

    return (
        temp.groupby("customer_id")
        .agg(
            **{
                f"spend_{label}": ("total_amount", "sum"),
                f"tx_{label}": ("transaction_id", "count")
            }
        )
        .reset_index()
    )

txn_31_60 = window_agg_safe(store_sales_header_df, 30, 60, "31_60d")
txn_61_120 = window_agg_safe(store_sales_header_df, 60, 120, "61_120d")
txn_121_180 = window_agg_safe(store_sales_header_df, 120, 180, "121_180d")


3.4 PRODUCT-LEVEL AGGREGATION

In [766]:
line_product_df = store_sales_line_items_df.merge(
    product_df,
    on="product_id",
    how="left"
).merge(
    store_sales_header_df[['transaction_id', 'customer_id']],
    on='transaction_id',
    how='left'
)


Category-level spend per customer

In [767]:
category_spend = (
    line_product_df
    .groupby(["customer_id", "product_category"])["line_item_amount"]
    .sum()
    .reset_index()
)


 Dominant category

In [768]:
dominant_category = (
    category_spend
    .sort_values("line_item_amount", ascending=False)
    .groupby("customer_id")
    .first()
    .reset_index()[["customer_id", "product_category"]]
    .rename(columns={"product_category": "dominant_category"})
)


Product diversity features

In [769]:
product_features = (
    line_product_df
    .groupby("customer_id")
    .agg(
        unique_products=("product_id", "nunique"),
        unique_categories=("product_category", "nunique")
    )
    .reset_index()
)


3.5 MERGE ALL CUSTOMER FEATURES

In [770]:
customer_features = (
    txn_agg
    .merge(txn_30, on="customer_id", how="left")
    .merge(txn_60, on="customer_id", how="left")
    .merge(txn_90, on="customer_id", how="left")
    .merge(product_features, on="customer_id", how="left")
    .merge(dominant_category, on="customer_id", how="left")
    .merge(
        customer_df[
            [
                "customer_id",
                "segment_id",
                "loyalty_status",
                "total_loyalty_points",
                "is_cold_start"
            ]
        ],
        on="customer_id",
        how="left"
    )
)


3.6 FINAL NULL HANDLING (POST-AGG)

In [771]:
# Customers with no recent activity → fill zeros
for col in customer_features.columns:
    if "spend_last" in col or "tx_last" in col:
        customer_features[col] = customer_features[col].fillna(0)

customer_features["dominant_category"] = customer_features[
    "dominant_category"
].fillna("Unknown")


FINAL CHECK

In [808]:
customer_features.to_csv("customer_features.csv", index=False)

In [772]:
print("✅ PHASE 3 COMPLETED")
print("Customer feature table shape:", customer_features.shape)

customer_features.head()


✅ PHASE 3 COMPLETED
Customer feature table shape: (20000, 21)


,customer_id,total_spend,total_transactions,avg_order_value,first_transaction_date,last_transaction_date,recency_days,tenure_days,spend_last_30d,tx_last_30d,...,tx_last_60d,spend_last_90d,tx_last_90d,unique_products,unique_categories,dominant_category,segment_id,loyalty_status,total_loyalty_points,is_cold_start
0,1,228156.76,19,12008.250526,2020-04-30,2021-08-02,1612,2071,0.0,0.0,...,0.0,0.0,0.0,53.0,8.0,Home,4.0,Gold,2347,0
1,2,14659.49,1,14659.490000,2022-10-23,2022-10-23,1165,1165,0.0,0.0,...,0.0,0.0,0.0,4.0,3.0,Beauty,4.0,Bronze,166,1
2,3,76364.73,5,15272.946000,2022-07-13,2022-10-23,1165,1267,0.0,0.0,...,0.0,0.0,0.0,15.0,7.0,Fashion,4.0,Bronze,763,0
3,4,64789.63,4,16197.407500,2019-09-24,2019-11-23,2230,2290,0.0,0.0,...,0.0,0.0,0.0,12.0,7.0,Fashion,3.0,Silver,663,0
4,5,218675.35,13,16821.180769,2022-08-15,2023-07-14,901,1234,0.0,0.0,...,0.0,0.0,0.0,43.0,8.0,Electronics,4.0,Silver,2186,0


In [807]:
customer_features.shape

(20000, 21)

In [773]:
customer_features.columns


Index(['customer_id', 'total_spend', 'total_transactions', 'avg_order_value',
       'first_transaction_date', 'last_transaction_date', 'recency_days',
       'tenure_days', 'spend_last_30d', 'tx_last_30d', 'spend_last_60d',
       'tx_last_60d', 'spend_last_90d', 'tx_last_90d', 'unique_products',
       'unique_categories', 'dominant_category', 'segment_id',
       'loyalty_status', 'total_loyalty_points', 'is_cold_start'],
      dtype='object')

Phase 4

In [774]:
cutoff_date = store_sales_header_df["transaction_date"].max() - pd.Timedelta(days=30)


4️⃣ FULL TARGET CREATION CODE

4.1 DEFINE CUTOFF & TARGET WINDOW

In [775]:
cutoff_date = store_sales_header_df["transaction_date"].max() - pd.Timedelta(days=30)
target_end_date = cutoff_date + pd.Timedelta(days=30)

print("Cutoff date:", cutoff_date)
print("Target window end:", target_end_date)


Cutoff date: 2025-12-01 00:00:00
Target window end: 2025-12-31 00:00:00


4.2 FILTER FUTURE TRANSACTIONS

In [776]:
future_tx = store_sales_header_df[
    (store_sales_header_df["transaction_date"] > cutoff_date) &
    (store_sales_header_df["transaction_date"] <= target_end_date)
]


4.3 COMPUTE TARGET PER CUSTOMER

In [777]:
target_30d = (
    future_tx
    .groupby("customer_id")["total_amount"]
    .sum()
    .reset_index()
    .rename(columns={"total_amount": "target_30d_spend"})
)


4.4 MERGE TARGET WITH FEATURES

In [778]:
final_dataset = customer_features.merge(
    target_30d,
    on="customer_id",
    how="left"
)


4.5 HANDLE CUSTOMERS WITH NO FUTURE SPEND

In [779]:
# Customers who made no purchase in next 30 days → spend = 0
final_dataset["target_30d_spend"] = final_dataset[
    "target_30d_spend"
].fillna(0)


INAL SANITY CHECKS (MANDATORY)

In [780]:
print("Final dataset shape:", final_dataset.shape)

print("\nTarget distribution:")
print(final_dataset["target_30d_spend"].describe())


Final dataset shape: (20000, 22)

Target distribution:
count    20000.000000
mean      1312.729566
std       5792.896434
min     -11563.680000
25%          0.000000
50%          0.000000
75%          0.000000
max      83747.440000
Name: target_30d_spend, dtype: float64


5.1 MOMENTUM & TREND FEATURES

In [781]:
final_dataset["spend_momentum_30_90"] = (
    final_dataset["spend_last_30d"] /
    (final_dataset["spend_last_90d"] + 1)
)

final_dataset["tx_momentum_30_90"] = (
    final_dataset["tx_last_30d"] /
    (final_dataset["tx_last_90d"] + 1)
)


Spend acceleration (recent vs older)

In [782]:
final_dataset["spend_acceleration"] = (
    final_dataset["spend_last_30d"] -
    (final_dataset["spend_last_90d"] - final_dataset["spend_last_30d"])
)


5.2 STABILITY & CONSISTENCY FEATURES

In [783]:
final_dataset["aov_stability"] = (
    final_dataset["avg_order_value"] /
    (final_dataset["total_spend"] / final_dataset["total_transactions"] + 1)
)


Purchase regularity

In [784]:
final_dataset["purchase_regularity"] = (
    final_dataset["tenure_days"] /
    (final_dataset["total_transactions"] + 1)
)


5.3 SEASONALITY FEATURES

In [785]:
final_dataset["cutoff_month"] = cutoff_date.month


Spend in same month last year

In [786]:
same_month_last_year = store_sales_header_df[
    (store_sales_header_df["transaction_date"].dt.month == cutoff_date.month) &
    (store_sales_header_df["transaction_date"].dt.year == cutoff_date.year - 1)
]

seasonal_spend = (
    same_month_last_year
    .groupby("customer_id")["total_amount"]
    .sum()
    .reset_index()
    .rename(columns={"total_amount": "spend_same_month_last_year"})
)

final_dataset = final_dataset.merge(
    seasonal_spend,
    on="customer_id",
    how="left"
)

final_dataset["spend_same_month_last_year"] = final_dataset[
    "spend_same_month_last_year"
].fillna(0)


5.4 LOYALTY & ENGAGEMENT INTERACTIONS

In [787]:
final_dataset["loyalty_intensity"] = (
    final_dataset["total_loyalty_points"] /
    (final_dataset["tenure_days"] + 1)
)


Loyalty × Recency interaction

In [788]:
final_dataset["loyalty_recency_interaction"] = (
    final_dataset["total_loyalty_points"] /
    (final_dataset["recency_days"] + 1)
)


5.5 CATEGORY DEPENDENCY FEATURES

In [789]:
final_dataset["category_dependency"] = (
    final_dataset["unique_categories"] /
    (final_dataset["unique_products"] + 1)
)


5.6 COLD-START REINFORCEMENT

In [790]:
final_dataset["cold_start_spend_proxy"] = np.where(
    final_dataset["is_cold_start"] == 1,
    final_dataset["avg_order_value"],
    final_dataset["spend_last_30d"]
)


5.7 SAFE LOG TRANSFORMS (IMPORTANT)

In [791]:
log_cols = [
    "total_spend",
    "spend_last_30d",
    "spend_last_60d",
    "spend_last_90d",
    "avg_order_value",
    "target_30d_spend"
]

for col in log_cols:
    final_dataset[f"log_{col}"] = np.log1p(final_dataset[col])


/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result 

5.8 FINAL FEATURE SANITY CHECK

In [792]:
final_dataset.replace([np.inf, -np.inf], 0, inplace=True)

print("✅ PHASE 5 FEATURE ENGINEERING COMPLETED")
print("Final dataset shape:", final_dataset.shape)


✅ PHASE 5 FEATURE ENGINEERING COMPLETED
Final dataset shape: (20000, 39)


PHASE 6: Train/validation split & modeling

In [ ]:
f

6.1 PREPARE MODELING DATA

In [793]:
TARGET = "target_30d_spend"

# Drop identifiers & leakage-prone columns
# Also drop raw date columns as 'recency_days' and 'tenure_days' already capture temporal information
DROP_COLS = [
    "customer_id",
    TARGET,
    "first_transaction_date",
    "last_transaction_date"
]

X = final_dataset.drop(columns=DROP_COLS)
y = final_dataset[TARGET]


 Encode categorical variables

In [794]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = X.select_dtypes(include=["object"]).columns
numeric_cols = X.select_dtypes(exclude=["object"]).columns


6.2 TIME-BASED TRAIN / VALIDATION SPLIT

In [795]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)



6.3 PREPROCESSING PIPELINE

In [796]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [797]:
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numeric_cols),
        ("cat", categorical_pipeline, categorical_cols)
    ]
)


6.5 ADVANCED MODEL (RECOMMENDED)

In [798]:
from sklearn.ensemble import GradientBoostingRegressor

gbr_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        random_state=42
    ))
])

gbr_model.fit(X_train, y_train)
gbr_pred = gbr_model.predict(X_val)


6.6 MODEL EVALUATION

In [799]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


In [800]:
def evaluate(y_true, y_pred, model_name):
    print(f"\n🔹 {model_name}")
    print("MAE :", mean_absolute_error(y_true, y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))
    print("R²  :", r2_score(y_true, y_pred))


In [801]:
evaluate(y_val, gbr_pred, "Gradient Boosting")


🔹 Gradient Boosting
MAE : 4.7196530268700885
RMSE: 76.35458859845541
R²  : 0.9998336001347062


In [802]:
# import joblib

# # Save the entire pipeline (preprocessor + model)
# joblib.dump(gbr_model, "gradient_boosting_30d_clv_model.pkl")


In [803]:
import joblib


In [804]:
MODEL_PATH = "gradient_boosting_30d_clv_model.pkl"

joblib.dump(gbr_model, MODEL_PATH)

print(f"✅ Model saved successfully at: {MODEL_PATH}")


✅ Model saved successfully at: gradient_boosting_30d_clv_model.pkl


In [805]:
# joblib.dump(gbr_model, "gradient_boosting_30d_clv_model.pkl")

In [806]:
# import joblib
# import os

# # ... (Your existing training code from above goes here) ...

# # ---------------------------------------------------------
# # SAVE THE MODEL
# # ---------------------------------------------------------
# model_filename = "gradient_boosting_30d_clv_model.pkl"

# # Save the pipeline (which includes the preprocessor and the model)
# joblib.dump(gbr_model, model_filename)

# print(f"✅ Model saved successfully to: {os.path.abspath(model_filename)}")
# print(f"   Filesize: {os.path.getsize(model_filename) / 1024:.2f} KB")

In [809]:
df = pd.read_csv('customer_features.csv')

In [810]:
df.head()

,customer_id,total_spend,total_transactions,avg_order_value,first_transaction_date,last_transaction_date,recency_days,tenure_days,spend_last_30d,tx_last_30d,...,tx_last_60d,spend_last_90d,tx_last_90d,unique_products,unique_categories,dominant_category,segment_id,loyalty_status,total_loyalty_points,is_cold_start
0,1,228156.76,19,12008.250526,2020-04-30,2021-08-02,1612,2071,0.0,0.0,...,0.0,0.0,0.0,53.0,8.0,Home,4.0,Gold,2347,0
1,2,14659.49,1,14659.490000,2022-10-23,2022-10-23,1165,1165,0.0,0.0,...,0.0,0.0,0.0,4.0,3.0,Beauty,4.0,Bronze,166,1
2,3,76364.73,5,15272.946000,2022-07-13,2022-10-23,1165,1267,0.0,0.0,...,0.0,0.0,0.0,15.0,7.0,Fashion,4.0,Bronze,763,0
3,4,64789.63,4,16197.407500,2019-09-24,2019-11-23,2230,2290,0.0,0.0,...,0.0,0.0,0.0,12.0,7.0,Fashion,3.0,Silver,663,0
4,5,218675.35,13,16821.180769,2022-08-15,2023-07-14,901,1234,0.0,0.0,...,0.0,0.0,0.0,43.0,8.0,Electronics,4.0,Silver,2186,0


In [811]:
df.columns

Index(['customer_id', 'total_spend', 'total_transactions', 'avg_order_value',
       'first_transaction_date', 'last_transaction_date', 'recency_days',
       'tenure_days', 'spend_last_30d', 'tx_last_30d', 'spend_last_60d',
       'tx_last_60d', 'spend_last_90d', 'tx_last_90d', 'unique_products',
       'unique_categories', 'dominant_category', 'segment_id',
       'loyalty_status', 'total_loyalty_points', 'is_cold_start'],
      dtype='object')